In [1]:
import numpy as np
import pandas as pd

import re
import nltk
import string
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TensorFlow Version: 2.13.0


In [2]:
import polars as pl

splits = {'train': '1.0.0/train-*.parquet', 'validation': '1.0.0/validation-00000-of-00001.parquet', 'test': '1.0.0/test-00000-of-00001.parquet'}
df = pl.read_parquet('hf://datasets/abisee/cnn_dailymail/' + splits['train'])


In [3]:
df = df.drop(['id'])
df = pd.DataFrame(df)

In [4]:
for i in range(5):
    print("Article #", i + 1)
    print('Original Text')
    print(df.iloc[i, 0])

    print('\n\n Summary Text:')
    print(df.iloc[i, 1])

    print('==' * 250)

Article # 1
Original Text
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box of

In [5]:
dfn=pd.DataFrame()
dfn['article'] = df[0]
dfn['summary'] = df[1]


In [6]:
dfn.isnull().sum()
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   article  287113 non-null  object
 1   summary  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


# Data preprocessing

In [7]:
# A list of contractions
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       287113 non-null  object
 1   1       287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [12]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   article  287113 non-null  object
 1   summary  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [ ]:
dfn['article'] = dfn['article'].apply(clean_text)
dfn['summary'] = dfn['summary'].apply(clean_text)

# Feature Engineering (Extractive Approach)

extracting important sentences using <br>
<b>Term Frequency (TF-IDF)</b><br>
<b>Sentence Position </b><br>
<b>Sentence length </b><br>
<b>Keyword Density</b><br>
<b>Named Entity Recognition (NER)</b><br>

#### Gyan ki baatein<br>
<b><i>Term Frequency (TF-IDF):  </b></i> Statistical measure used to evaluate importance of a word in document relative to coupus of documents , it gives out scores <br>

<b><i>Sentence Position</b></i> Sentences at beginning and at end of the document often carry significant weight , assigning higher scores based on their position <br>

<b><i>Sentence Length</b></i> as Longer sentences carry more info , so using length as a scoring factor<br>

<b><i>Keyword Density </b></i> using keyword density to identify sentences with high concentration of important words

<b><i>Named Entity Recognition (NER) </b></i> NER helps identify and categorize key entities like names , organization , locations etc , Using spaCy to perform NER<br>

#### Sequence to be applied
<b><i>Named Entity Recognition (NER)</b></i> 

<b><i>Term Frequency (TF-IDF)</b></i> 

<b><i>Sentence Position</b></i> 

<b><i>Keyword Density</b></i> 

<b><i>Sentence Length</b></i> 

the following code block cleans and then applies feature extraction procedure properly

In [14]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
from nltk.data import find
find('taggers/averaged_perceptron_tagger')

FileSystemPathPointer('C:\\Users\\Shiva\\AppData\\Roaming\\nltk_data\\taggers\\averaged_perceptron_tagger')

In [ ]:
dfn['article_features']

In [ ]:
import pandas as pd
import polars as pl

# Assuming you already have a huge Pandas dataframe called df
pl_df = pl.from_pandas(dfn)
pl_df.write_csv("output.csv")


Fatser Text preprocessing

In [ ]:
import spacy
import re
import torch
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from torch.utils.data import Dataset , dataloader

#load spacy with GPU (if available)
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm' , disable=['parser' , 'tagger']) #keep NER , lemmatizer

#configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 256
MAX_SENTENCES = 30 #truncate long documents

#custom stopwords 
STOPWORDS = nlp.Defaults.stop_words | {"'s" , "n't" , "'m" , "'re"}

#--------------------------------------------------------------------------------
#GPU-Optimized text preprocessing
#--------------------------------------------------------------------------------
def preprocess_batch(texts):
    '''Process 1000 documents at once on GPU'''
    cleaned = []
    for doc in nlp.pipe(texts , batch_size=1000):
        sentences = []
        for sent in doc.sents:
            tokens=[
                token.lemma_.lower() for token in sent
                if not token.is_stop and token.lemma_ not in STOPWORDS
            ]
            sentences.append(' '.join(tokens))
            cleaned.append(sentences[:MAX_SENTENCES]) #TRUNCATE
    return cleaned

#--------------------------------------------------------------------------------------
#Feature Engineering (GPU Tensors)
#--------------------------------------------------------------------------------------
class FeatureExtractor:
    def __inti__(self):
        self.global_freq = defaultdict(int)

    def fit(self , all_sentences):
        '''BUILD global vocabulary'''
        for sentences in tqdm(all_sentences):
            for sent in sentences:
                for token in sent.split():
                    self.global_freq[token] +=1

    def transform(self , sentences):
        '''Convert sentences to Pytorch feature tensors'''
        features = []
        for i , sent in enumerate(sentences):
            tokens = sent.split()
            if not tokens:
                continue

            #TF-IDF Calculation (GPU)
            tf = torch.tensor(
                [tokens.count(word)/len(tokens) for word in tokens]
            )



SyntaxError: incomplete input (513217508.py, line 46)